<a href="https://colab.research.google.com/github/bartek717/projectx/blob/main/random_baseline_FLAN_T5_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, BertTokenizer, BertModel
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from google.colab import files
import time
import math
import os
# here you can change the model if you want. Looks like the available options are (flan-t5-) small, base, large, xl, xxl. Probably easiest to use one of the smaller models for our use case.
import random


from google.colab import drive
drive.mount('/content/drive')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.4 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# subpubmed or whatever preprocessing
!git clone https://github.com/vgupta123/sumpubmed.git
%cd sumpubmed/

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 23.59 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.
/content/sumpubmed


In [ ]:
def process_data(text_dir, summary_dir):
    texts = []
    summaries = []

    # Assuming the file names in both directories match
    for filename in os.listdir(text_dir):
        if filename.endswith(".txt"):
            # Read the full text
            with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

            # Read the corresponding summary
            summary_file = "abst_"+(filename.split("_")[-1])
            with open(os.path.join(summary_dir, summary_file), 'r', encoding='utf-8') as file:
                summaries.append(file.read())

    return texts, summaries


In [ ]:
texts_full, summaries_full = process_data('line_text/', 'shorter_abstract/')

In [ ]:
def clean_text(text):
    # Replace <cit> and <dig> with an empty string
    cleaned_text = text.replace("<cit>", "").replace("<dig>", "")
    return cleaned_text

In [ ]:
cleaned_texts = [clean_text(text) for text in texts_full]
cleaned_summaries = [clean_text(summary) for summary in summaries_full]

In [ ]:
texts = cleaned_texts[:int(0.85*len(cleaned_texts))]
summaries = cleaned_summaries[:int(0.85*len(cleaned_summaries))]

remaining_texts = texts[int(0.85*len(texts)):]
remaining_summaries = summaries[int(0.85*len(summaries)):]

In [ ]:
# BEGINNING OF RANDOM SELECTION

In [ ]:
def getRandomSummaries(percentage):
    num_samples = int(percentage * len(texts))
    indices = random.sample(range(len(texts)), num_samples)
    selected_texts = [texts[i] for i in indices]
    selected_summaries = [summaries[i] for i in indices]

    return selected_texts, selected_summaries

In [ ]:
# 70, 50, 30, 10, 5, 3, 1

percents = [0.70, 0.50, 0.30, 0.10, 0.05, 0.03, 0.01]
selected_summaries=[]
selected_texts = []

for i in percents:
    t, s = getRandomSummaries(i)
    selected_summaries.append(s)
    selected_texts.append(t)


print('original: ' + str(len(texts)))
for i in selected_summaries:
  print('new: :' + str(len(i)))


original: 27785
new: :19449
new: :13892
new: :8335
new: :2778
new: :1389
new: :833
new: :277


In [ ]:
# END OF CORESET SELECTION

In [ ]:
def tokenize_function(examples):
    # Tokenize the inputs and labels
    tokenized_inputs = tokenizer(examples["texts"], padding=True, truncation=True, max_length=1024)
    with tokenizer.as_target_tokenizer():
        tokenized_labels = tokenizer(examples["summaries"], padding=True, truncation=True, max_length=256)

    tokenized_inputs["labels"] = tokenized_labels["input_ids"]
    return tokenized_inputs

In [ ]:
def train_model(model, train_dataset, eval_dataset, tokenizer, percent):
    start = time.time()
    # we need to standardize the training arguments
    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=3,
        predict_with_generate=True,
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    trainer.train()
    end = time.time()
    elapsed_time = end - start
    evaluation_results = trainer.evaluate()

    model_output_dir = '/content/drive/My Drive/t5finerandom_' + str(percent)[-2:]
    tokenizer_output_dir = '/content/drive/My Drive/t5finerandom_' + str(percent)[-2:]
    model.save_pretrained(model_output_dir)
    tokenizer.save_pretrained(tokenizer_output_dir)

    return time, evaluation_results, model_output_dir, tokenizer_output_dir

In [ ]:
for i in range(len(selected_summaries)):
    model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
    tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    summary = selected_summaries[i]
    text = selected_texts[i]
    print('percent: ' + str(percents[i]))
    print('summary length: ' + str(len(summary)))
    print('text length: ' + str(len(text)))
    dataset = Dataset.from_dict({"texts": text, "summaries": summary})
    total_examples = len(dataset)
    print('total examples: ' + str(len(dataset)))
    train_size = int(0.95 * total_examples)
    eval_and_test_size = (total_examples - train_size)
    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_datasets.select(range(train_size))
    eval_dataset = tokenized_datasets.select(range(train_size, train_size+eval_and_test_size))


    time, evaluation_results, model_output_dir, tokenizer_output_dir = train_model(model, train_dataset, eval_dataset, tokenizer, percents[i])
    print(time)
    print(evaluation_results)
    print(model_output_dir)
    print(tokenizer_output_dir)
    print('-'*100)

percent: 0.7
summary length: 19449
text length: 19449
total examples: 19449


Map:   0%|          | 0/19449 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.768600,2.525283
2,2.682800,2.486931
3,2.653100,2.481452


<module 'time' (built-in)>
{'eval_loss': 2.4814517498016357, 'eval_runtime': 15.6427, 'eval_samples_per_second': 62.202, 'eval_steps_per_second': 31.133, 'epoch': 3.0}
/content/drive/My Drive/t5finerandom_.7
/content/drive/My Drive/t5finerandom_.7
----------------------------------------------------------------------------------------------------
percent: 0.5
summary length: 13892
text length: 13892
total examples: 13892


...

<module 'time' (built-in)>
{'eval_loss': 2.7510976791381836, 'eval_runtime': 0.6768, 'eval_samples_per_second': 62.061, 'eval_steps_per_second': 31.031, 'epoch': 3.0}
/content/drive/My Drive/t5finerandom_03
/content/drive/My Drive/t5finerandom_03
----------------------------------------------------------------------------------------------------
percent: 0.01
summary length: 277
text length: 277
total examples: 277


Map:   0%|          | 0/277 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3856: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,4.212804
2,No log,3.796881
3,No log,3.635277


<module 'time' (built-in)>
{'eval_loss': 3.6352767944335938, 'eval_runtime': 0.2285, 'eval_samples_per_second': 61.267, 'eval_steps_per_second': 30.634, 'epoch': 3.0}
/content/drive/My Drive/t5finerandom_01
/content/drive/My Drive/t5finerandom_01
----------------------------------------------------------------------------------------------------
